In [1]:
import tensorflow as tf
from archs.segmentation.unet import build_unet
from data.data_generator import DataGenerator
from data.pre_processor import PreProcessor
from utils.loss import FocalDiceLoss, dice_coef
from data.augmentor import Augmentor
from model_utils import custom_loss
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np

import os
np.random.seed(2023)  # Set seed for reproducibility
tf.random.set_seed(2026) # Note: Different to test different initializations.
!pip install tqdm # Adding tqdm to use progress bars. Unbarable waiting for each epoch to finish without feedback.
from tqdm import tqdm

2023-03-24 14:34:09.890413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 14:34:10.463682: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/theodor/anaconda3/envs/tf/lib/
2023-03-24 14:34:10.463754: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/theodor/anaconda3/envs/tf/lib/
2023-03-24 14:34:10.463761: W tensorflow/compiler/tf2t

In [2]:

gen_dir = './data_zip/' # Change if you have copied the data locally on your machine 
array_labels = ['t1', 't1ce', 't2', 'flair','mask']  # Available arrays are: 't1', 't1ce', 't2', 'flair', 'mask'.
per_batch = len(array_labels)-1 # We don't want to use the mask as an input.
batch_size = 16

batch_size = batch_size//per_batch 
W,H,C = 256,256,1
input_shape = (W,H,C)
num_classes = 1

gen_train = DataGenerator(data_path=gen_dir + 'training',
                          arrays=array_labels,
                          batch_size=batch_size)

gen_val = DataGenerator(data_path=gen_dir + 'validating',
                        arrays=array_labels,
                        batch_size=batch_size)

gen_test = DataGenerator(data_path=gen_dir + 'testing',
                         arrays=array_labels,
                         batch_size=batch_size)
augmentor = Augmentor(translate=0, # No translation. Due to lack of speed.
                      shear=0, # No shear. Due to lack of speed.
                      rotate=0, # No rotation. Due to lack of speed.
                      mask=0.8, # Probability of masking the image.
                      mask_size=0.2, # Maximum size of the mask as a fraction of the image size.
                      max_n_masks=8, # Maximum number of masks to apply.
                      noise=0.4, # Probability of adding Gaussian noise to the image.
                      noise_mean=0.05, # Mean of the noise.
                      noise_std=0.1, # Standard deviation of the noise.
                      ) # Augmentation of the data.
pre_processor = PreProcessor(W,H,C,batch_size,per_batch,augmentor)

In [3]:
from archs.segmentation.attention_encoders import VisualAttention
atten_model = VisualAttention(d_model=756, num_heads=6,

Model: "visual_attention"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 patch_embedding (PatchEmbed  multiple                 590592    
 ding)                                                           
                                                                 
 multi_head_attention (Multi  multiple                 12595968  
 HeadAttention)                                                  
                                                                 
 dense (Dense)               multiple                  393728    
                                                                 
 layer_normalization (LayerN  multiple                 1024      
 ormalization)                                                   
                                                                 
Total params: 13,581,312
Trainable params: 13,581,312
Non-trainable params: 0
______________________________________

2023-03-24 14:34:56.156151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 14:34:56.156448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 14:34:56.156673: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 14:34:56.157062: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 14:34:56.157292: I tensorflow/compiler/xla/stream_executo

In [ ]:
filters = [8,16,32,64,128,256,512,1024]
kernel_size = [3,3,3,3,3,3,3,1]
strides = 1
padding = "same"
activation = "selu"
# drop_rate_encoder = [0.01,0.02,0.02,0.1]
# drop_rate_decoder = [0.01,0.1,0.1,0]
drop_rate_encoder = [0.0,0.02,0.02,0.05]
drop_rate_decoder = [0.0]
depth_encoder = [2,2,3,4,4,5,6,8]
depth_decoder = [1,1,1,1,1,1,1,1]
output_depth = 10
output_activation = "sigmoid"

model = build_unet(
    input_shape=input_shape,
    num_classes=num_classes,
    filters=filters,
    kernel_size=kernel_size,
    strides=strides,
    padding=padding,
    activation=activation,
    depth_encoder=depth_encoder,
    decoder_type="add",
    upsample_type="bilinear",
    depth_decoder=depth_decoder,
    drop_rate_encoder=drop_rate_encoder,
    drop_rate_decoder=drop_rate_decoder,
    output_depth=output_depth,
    output_activation=output_activation,
)
model.summary()